In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

data = pd.read_csv('final.csv')
#map(complex, dat['theta_fft'])

data = data.drop(['theta_fft', 'alpha_low_fft', 'alpha_high_fft', 'beta_fft', 'gamma_fft'], axis=1)

data = data.sample(frac=1)

print(data)

        theta_mean  theta_median     theta_std     theta_var  alpha_low_mean  \
3323  1.216771e-13  6.699750e-14  9.171290e-14  8.411256e-27    9.356620e-14   
3158  3.206124e-13  3.325870e-13  1.313813e-13  1.726103e-26    2.034800e-13   
974   5.942312e-14  4.617329e-14  2.945417e-14  8.675479e-28    5.939985e-14   
1073  1.458082e-14  1.653744e-14  5.213257e-15  2.717804e-29    1.506904e-14   
447   3.270426e-15  3.634977e-15  1.603246e-15  2.570397e-30    4.483288e-15   
3177  2.348545e-14  2.102542e-14  1.025795e-14  1.052255e-28    4.648655e-14   
1554  4.584316e-13  3.298683e-13  2.690379e-13  7.238139e-26    1.846134e-13   
4415  1.062250e-14  8.447134e-15  4.809650e-15  2.313273e-29    1.327124e-14   
1453  1.953286e-13  1.392763e-13  1.522527e-13  2.318089e-26    1.782102e-13   
2249  3.476132e-14  3.084865e-14  1.249630e-14  1.561574e-28    3.586565e-14   
4169  2.398198e-12  1.109162e-12  2.480020e-12  6.150497e-24    1.905171e-12   
2604  4.249323e-14  4.018491e-14  3.0419

In [49]:
X_data = data.drop('state', axis=1)
X_data.shape
y_data = data['state']
y_data.shape

(5043,)

In [50]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X_data, y_data, random_state=1)

In [51]:
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

In [52]:
from scipy.stats import randint

rf_grid = {
'n_estimators': [100, 200, 1000],
'max_features': [1, "sqrt"],
'max_depth': randint(1, 50),
'min_samples_split': randint(2, 15),
'min_samples_leaf': randint(1, 15),
'warm_start': [True, False],
'class_weight': ["balanced", None],
'random_state': [1]
}

In [53]:
from sklearn.model_selection import RandomizedSearchCV, ShuffleSplit

cv = ShuffleSplit(n_splits=1,test_size=0.2)
best_score = 0
scores = []
for i in range(10):
    clf = RandomizedSearchCV(RandomForestClassifier(), rf_grid, n_jobs=-1, cv=cv, scoring = 'accuracy')
    %time clf.fit(Xtrain[:], ytrain[:])
    scores.append(clf.best_score_)
    if best_score < clf.best_score_:
        et_params = clf.best_params_
        best_score = clf.best_score_
print ('Mean: ', np.mean(scores))
print ('Std: ', np.std(scores))
print (clf.best_params_)
print (clf.best_score_)

Wall time: 35.9 s
Wall time: 10.7 s
Wall time: 39.3 s
Wall time: 15.6 s
Wall time: 7.76 s
Wall time: 31.8 s
Wall time: 35.6 s


C:\Users\danic\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '


Wall time: 41.4 s
Wall time: 35 s
Wall time: 20.9 s
Mean:  0.6021136063408191
Std:  0.009448641871729288
{'class_weight': 'balanced', 'max_depth': 30, 'max_features': 1, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 1000, 'random_state': 1, 'warm_start': False}
0.607661822985469


In [54]:
rf_params = clf.best_params_
clf = RandomForestClassifier().set_params(**rf_params).fit(Xtrain, ytrain)
# Predict results
pred = clf.predict(Xtest)

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print ("Confusion Matrix: ")
print (confusion_matrix(pred, ytest))

print ("\n Classification Report: ")
print (classification_report(pred,ytest))

print ("\nAccuracy: ", accuracy_score(pred, ytest))
clf.feature_importances_

Confusion Matrix: 
[[ 98  19  69]
 [ 28 373 128]
 [118 120 308]]

 Classification Report: 
              precision    recall  f1-score   support

       bored       0.40      0.53      0.46       186
        calm       0.73      0.71      0.72       529
  frustrated       0.61      0.56      0.59       546

   micro avg       0.62      0.62      0.62      1261
   macro avg       0.58      0.60      0.59      1261
weighted avg       0.63      0.62      0.62      1261


Accuracy:  0.6177636796193497


array([0.0472801 , 0.05000586, 0.05040897, 0.05155375, 0.04585075,
       0.04647603, 0.05180074, 0.05052442, 0.04592421, 0.04888807,
       0.05131366, 0.05196474, 0.04801068, 0.04839597, 0.05479869,
       0.05350124, 0.04827296, 0.04785921, 0.05327546, 0.0538945 ])